## Downloading Hadoop

In [1]:
%env TARGET_HADOOP_VERSION hadoop-2.9.2

env: TARGET_HADOOP_VERSION=hadoop-2.9.2


In [2]:
!wget http://ftp.unicamp.br/pub/apache/hadoop/common/${TARGET_HADOOP_VERSION}/${TARGET_HADOOP_VERSION}.tar.gz -q --show-progress

hadoop-2.9.2.tar.gz 100%[===================>] 349.47M  4.93MB/s    in 32s     


### Extracting compressed files

Optional: removing tar file

In [3]:
!tar -xvf ${TARGET_HADOOP_VERSION}.tar.gz >/dev/null 
!rm       ${TARGET_HADOOP_VERSION}.tar.gz

### Discovering Java path

In [4]:
!dirname $(dirname $(readlink -f $(which javac)))

/usr/lib/jvm/java-8-openjdk-amd64


### Setting Java path envvar

We also added it to user's .bashr so it will be loaded as nodes perform ssh connections.

In [7]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [8]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " > .bashrc

# Using Hadoop in Standalone Mode (local)

# Using Hadoop in Pseudo-Distributed Mode

### Starting sshd server

Check postBuild and sshd_config files for more details

In [9]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding to know hosts by establishing a ssh connectcion (avoiding yes/no host confirmation)

In [10]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 8822 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 8822 -C "exit"

### Copying configurations files to Hadoop folder

In [11]:
!cp resources/configs/hadoop/* ${TARGET_HADOOP_VERSION}/etc/hadoop/

### Formating the filesystem

In [19]:
!./${TARGET_HADOOP_VERSION}/bin/hdfs namenode -format -force -nonInteractive

### Adding ssh options: running in a diferent port

In [13]:
%env HADOOP_SSH_OPTS -p 8822 

env: HADOOP_SSH_OPTS=-p 8822


### Starting NameNode daemon and DataNode daemon

In [20]:
# ! ./${TARGET_HADOOP_VERSION}/sbin/stop-dfs.sh
! ./${TARGET_HADOOP_VERSION}/sbin/start-dfs.sh

Starting namenodes on [localhost]
localhost: namenode running as process 1899. Stop it first.
localhost: datanode running as process 1989. Stop it first.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: secondarynamenode running as process 2168. Stop it first.


### Creating a folder in the distributed file system

In [23]:
!./${TARGET_HADOOP_VERSION}/bin/hdfs dfs -mkdir /user
!./${TARGET_HADOOP_VERSION}/bin/hdfs dfs -mkdir /user/matheus

mkdir: `/user': File exists
mkdir: `/user/matheus': File exists


### Copying a file to a folder in the distributed file system

In [27]:
!./${TARGET_HADOOP_VERSION}/bin/hdfs dfs -put readme.md /user/matheus

In [25]:
!./${TARGET_HADOOP_VERSION}/bin/hdfs dfs -ls /user/matheus

# YARN on a Single Node

In [26]:
!./${TARGET_HADOOP_VERSION}/sbin/start-yarn.sh

starting yarn daemons
resourcemanager running as process 2478. Stop it first.
localhost: nodemanager running as process 2567. Stop it first.
